# Install packages

# download all the dependencies that we will need

In [1]:
import numpy as np
import pandas as pd

!pip install folium
import folium 

from geopy.geocoders import Nominatim 

import requests

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('USdata.csv')
df.head()

,Postcode,Neighbourhood,Borough,Latitude,Longitude
0,501,Holtsville,NY,40.81,-73.04
1,544,Holtsville,NY,40.81,-73.04
2,601,Adjuntas,PR,18.16,-66.72
3,602,Aguada,PR,18.38,-67.18
4,603,Aguadilla,PR,18.43,-67.15


# get coordinate of State of RI

In [3]:
RI_df = df[df.Borough == 'RI'].reset_index(drop=True)
RI_df.shape

(91, 5)

In [4]:
loc = 'Rhode Island,US'

geolocator = Nominatim(user_agent="ny_explorer")
RI = geolocator.geocode(loc)
RI_lat = RI.latitude
RI_long = RI.longitude
print('The geograpical coordinate of Rhode Island,US are {}, {}.'.format(RI_lat, RI_long))

The geograpical coordinate of Rhode Island,US are 41.7962409, -71.5992372.


In [5]:
RI_map = folium.Map(location = [RI_lat, RI_long], 
                                  zoom_start = 10)

for lat, lng, label in zip(RI_df.Latitude, 
                           RI_df.Longitude, 
                           RI_df.Neighbourhood):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True).add_to(RI_map)  
    
RI_map

# Define Foursquare Credentials and Version

In [6]:
CLIENT_ID = 'J5ZDPTMLLUQEDHWC0LCKFRU55C0VLCC0BKH4AVNMZJOMP2DP' # your Foursquare ID
CLIENT_SECRET = 'SOEAIYU1TXL3UKM1IKAR1UGGIVFN531R1YRQXE0ZXQKL4AZG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J5ZDPTMLLUQEDHWC0LCKFRU55C0VLCC0BKH4AVNMZJOMP2DP
CLIENT_SECRET:SOEAIYU1TXL3UKM1IKAR1UGGIVFN531R1YRQXE0ZXQKL4AZG


# Explore Neighborhoods in RI
# create the GET request URL

In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius = 500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["categories"][0]["name"]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Neighbourhood", 
                  "Neighbourhood Latitude", 
                  "Neighbourhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Category"]
    
    return(nearby_venues)

In [8]:
LIMIT = 100
RI_venues = getNearbyVenues(names = RI_df.Neighbourhood, 
                               latitudes = RI_df.Latitude, 
                               longitudes = RI_df.Longitude)

In [9]:
RI_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Albion,41.95,-71.46,Kirkbrae Country Club,41.946065,-71.462193,Golf Course
1,Albion,41.95,-71.46,Twisted Hop Tavern,41.951100,-71.455864,Restaurant
2,Albion,41.95,-71.46,AnziTech Computer & Network Service,41.952948,-71.463189,IT Services
3,Albion,41.95,-71.46,albion corner store,41.951623,-71.454527,Convenience Store
4,Ashaway,41.42,-71.78,Pelloni Farm Market,41.419584,-71.778261,Farmers Market


In [10]:
print(RI_venues.shape)

print('The dataframe has {} neighbourhoods.'.format(
        len(RI_venues.Neighbourhood.unique())
    )
)

(740, 7)
The dataframe has 44 neighbourhoods.


In [11]:
print('There are {} uniques categories.'.format(len(RI_venues['Venue Category'].unique())))

There are 150 uniques categories.


In [12]:
# one hot encoding
RI_onehot = pd.get_dummies(RI_venues[['Venue Category']], 
                                         prefix="", 
                                         prefix_sep="")

# add neighborhood column back to dataframe
RI_onehot['Neighbourhood'] = RI_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [RI_onehot.columns[-1]] + list(RI_onehot.columns[:-1])
RI_onehot = RI_onehot[fixed_columns]

RI_onehot.head()

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,...,Thai Restaurant,Theater,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Store,Wine Bar,Wine Shop,Yoga Studio
0,Albion,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Albion,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Albion,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Albion,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ashaway,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
RI_grouped = RI_onehot.groupby('Neighbourhood').mean().reset_index()
RI_grouped

,Neighbourhood,American Restaurant,Art Gallery,Art Museum,Asian Restaurant,Assisted Living,Athletics & Sports,Australian Restaurant,BBQ Joint,Bagel Shop,...,Thai Restaurant,Theater,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Store,Wine Bar,Wine Shop,Yoga Studio
0,Albion,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,Ashaway,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Block Island,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bradford,0.000000,0.000000,0.000000,0.000000,0.000000,0.50,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bristol,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Falls,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
6,Charlestown,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
7,Chepachet,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
8,Coventry,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
9,Cranston,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
num_top_venues = 8

for hood in RI_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = RI_grouped[RI_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion----
                       venue  freq
0          Convenience Store  0.25
1                 Restaurant  0.25
2                IT Services  0.25
3                Golf Course  0.25
4  Middle Eastern Restaurant  0.00
5         Miscellaneous Shop  0.00
6                      Motel  0.00
7                     Museum  0.00


----Ashaway----
                       venue  freq
0             Farmers Market   1.0
1        American Restaurant   0.0
2   Mediterranean Restaurant   0.0
3  Middle Eastern Restaurant   0.0
4         Miscellaneous Shop   0.0
5                      Motel   0.0
6                     Museum   0.0
7                Music Venue   0.0


----Block Island----
                       venue  freq
0                       Food  0.33
1             Clothing Store  0.33
2                       Lake  0.33
3                  Nightclub  0.00
4  Middle Eastern Restaurant  0.00
5         Miscellaneous Shop  0.00
6                      Motel  0.00
7                     Museum  0.00

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns = columns)
neighbourhoods_venues_sorted['Neighbourhood'] = RI_grouped['Neighbourhood']

for ind in np.arange(RI_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(RI_grouped.iloc[ind, :], 
                                                                           num_top_venues)

neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albion,Golf Course,Convenience Store,IT Services,Restaurant,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
1,Ashaway,Farmers Market,Yoga Studio,Food,Fast Food Restaurant,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
2,Block Island,Food,Lake,Clothing Store,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
3,Bradford,Speakeasy,Athletics & Sports,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
4,Bristol,Pizza Place,Dive Bar,Breakfast Spot,Bakery,Pharmacy,Café,Mediterranean Restaurant,Liquor Store,Seafood Restaurant,Bar
5,Central Falls,Liquor Store,Restaurant,Convenience Store,Donut Shop,Optical Shop,Discount Store,Gas Station,Falafel Restaurant,Dive Bar,Diner
6,Charlestown,Home Service,Park,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
7,Chepachet,Trail,Yoga Studio,Department Store,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
8,Coventry,Bar,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Dessert Shop
9,Cranston,Playground,Business Service,Baseball Field,Pharmacy,Deli / Bodega,Liquor Store,Fried Chicken Joint,Skating Rink,Pizza Place,Cosmetics Shop


In [17]:
# set number of clusters
kclusters = 8

RI_grouped_clustering = RI_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, 
                random_state = 0).fit(RI_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 5, 7, 1], dtype=int32)

In [18]:
RI_merged = RI_df

# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
RI_merged = RI_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), 
                                                       on = 'Neighbourhood')

RI_merged.head() # check the last columns!


,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2801,Adamsville,RI,41.51,-71.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2802,Albion,RI,41.95,-71.46,1.0,Golf Course,Convenience Store,IT Services,Restaurant,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
2,2804,Ashaway,RI,41.42,-71.78,1.0,Farmers Market,Yoga Studio,Food,Fast Food Restaurant,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
3,2806,Barrington,RI,41.73,-71.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2807,Block Island,RI,41.16,-71.58,1.0,Food,Lake,Clothing Store,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner


In [19]:
RI_merged.dropna(subset=['Cluster Labels'], inplace = True)
RI_merged

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,2802,Albion,RI,41.95,-71.46,1.0,Golf Course,Convenience Store,IT Services,Restaurant,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
2,2804,Ashaway,RI,41.42,-71.78,1.0,Farmers Market,Yoga Studio,Food,Fast Food Restaurant,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
4,2807,Block Island,RI,41.16,-71.58,1.0,Food,Lake,Clothing Store,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
5,2808,Bradford,RI,41.39,-71.75,1.0,Speakeasy,Athletics & Sports,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
6,2809,Bristol,RI,41.67,-71.27,1.0,Pizza Place,Dive Bar,Breakfast Spot,Bakery,Pharmacy,Café,Mediterranean Restaurant,Liquor Store,Seafood Restaurant,Bar
8,2813,Charlestown,RI,41.38,-71.65,1.0,Home Service,Park,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
9,2814,Chepachet,RI,41.91,-71.70,5.0,Trail,Yoga Studio,Department Store,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
11,2816,Coventry,RI,41.68,-71.66,7.0,Bar,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Dessert Shop
13,2818,East Greenwich,RI,41.63,-71.50,1.0,Other Repair Shop,Tree,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
14,2822,Exeter,RI,41.56,-71.68,1.0,Gun Shop,Yoga Studio,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner


In [20]:
RI_merged['Cluster Labels'] = RI_merged['Cluster Labels'].astype(int)

In [21]:
RI_merged

,Postcode,Neighbourhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,2802,Albion,RI,41.95,-71.46,1,Golf Course,Convenience Store,IT Services,Restaurant,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
2,2804,Ashaway,RI,41.42,-71.78,1,Farmers Market,Yoga Studio,Food,Fast Food Restaurant,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
4,2807,Block Island,RI,41.16,-71.58,1,Food,Lake,Clothing Store,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
5,2808,Bradford,RI,41.39,-71.75,1,Speakeasy,Athletics & Sports,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
6,2809,Bristol,RI,41.67,-71.27,1,Pizza Place,Dive Bar,Breakfast Spot,Bakery,Pharmacy,Café,Mediterranean Restaurant,Liquor Store,Seafood Restaurant,Bar
8,2813,Charlestown,RI,41.38,-71.65,1,Home Service,Park,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
9,2814,Chepachet,RI,41.91,-71.70,5,Trail,Yoga Studio,Department Store,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
11,2816,Coventry,RI,41.68,-71.66,7,Bar,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Dessert Shop
13,2818,East Greenwich,RI,41.63,-71.50,1,Other Repair Shop,Tree,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
14,2822,Exeter,RI,41.56,-71.68,1,Gun Shop,Yoga Studio,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner


In [22]:
# create map
map_clusters = folium.Map(location = [RI_lat, RI_long], zoom_start = 10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(RI_merged['Latitude'], 
                                  RI_merged['Longitude'], 
                                  RI_merged['Neighbourhood'], 
                                  RI_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, lon],
        radius = 7,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster-1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

In [23]:
RI_merged.loc[RI_merged['Cluster Labels'] == 0, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Saunderstown,0,Sports Club,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Department Store


In [24]:
RI_merged.loc[RI_merged['Cluster Labels'] == 1, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Albion,1,Golf Course,Convenience Store,IT Services,Restaurant,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
2,Ashaway,1,Farmers Market,Yoga Studio,Food,Fast Food Restaurant,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
4,Block Island,1,Food,Lake,Clothing Store,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
5,Bradford,1,Speakeasy,Athletics & Sports,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store
6,Bristol,1,Pizza Place,Dive Bar,Breakfast Spot,Bakery,Pharmacy,Café,Mediterranean Restaurant,Liquor Store,Seafood Restaurant,Bar
8,Charlestown,1,Home Service,Park,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
13,East Greenwich,1,Other Repair Shop,Tree,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
14,Exeter,1,Gun Shop,Yoga Studio,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
20,Greenville,1,Breakfast Spot,Bistro,Convenience Store,Pizza Place,Dance Studio,Ice Cream Shop,Spa,Restaurant,Discount Store,Farmers Market
24,Hope Valley,1,American Restaurant,Bakery,Liquor Store,Sporting Goods Shop,Construction & Landscaping,Gym,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop


In [25]:
RI_merged.loc[RI_merged['Cluster Labels'] == 2, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,West Warwick,2,Snack Place,Dessert Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Yoga Studio


In [26]:
RI_merged.loc[RI_merged['Cluster Labels'] == 3, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Wakefield,3,Basketball Court,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Dessert Shop
54,Wakefield,3,Basketball Court,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Dessert Shop
80,North Providence,3,Basketball Court,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Dessert Shop


In [27]:
RI_merged.loc[RI_merged['Cluster Labels'] == 4, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Warren,4,Nightlife Spot,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Department Store


In [28]:
RI_merged.loc[RI_merged['Cluster Labels'] == 5, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Chepachet,5,Trail,Yoga Studio,Department Store,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
43,Cumberland,5,Cemetery,Trail,Department Store,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop
47,Rockville,5,Campground,Trail,Yoga Studio,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop


In [29]:
RI_merged.loc[RI_merged['Cluster Labels'] == 6, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Shannock,6,Playground,Yoga Studio,Department Store,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner,Dessert Shop


In [30]:
RI_merged.loc[RI_merged['Cluster Labels'] == 7, RI_merged.columns[[1] + list(range(5, RI_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Coventry,7,Bar,Yoga Studio,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Dessert Shop
67,Woonsocket,7,Outdoors & Recreation,Bar,Yoga Studio,Dessert Shop,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Discount Store,Diner
87,Johnston,7,Burger Joint,Bar,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Donut Shop,Dive Bar,Diner
